# 코드셀 0 - Emotion-FAN GitHub에서 코드 다운로드

In [1]:
!git clone https://github.com/Open-Debin/Emotion-FAN.git

fatal: destination path 'Emotion-FAN' already exists and is not an empty directory.


In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 25.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# 코드셀 1 Custom Dataset 클래스 정의

In [1]:
from torchvision import datasets, transforms
from PIL import Image, ImageOps
import numpy as np
import os
from ultralytics import YOLO

# YOLOv8 얼굴 탐지 모델 로딩 (한 번만 실행)
yolo_model = YOLO("yolov8n-face.pt")

class CustomEmotionFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None, face_detector=None):
        valid_exts = ('.jpg', '.jpeg', '.png')

        def has_valid_extension(path):
            return path.lower().endswith(valid_exts)

        super().__init__(root, transform=transform, is_valid_file=has_valid_extension)
        self.class_mapping = self.class_to_idx
        self.face_detector = face_detector
        print("✅ class_to_idx:", self.class_mapping)

    def __getitem__(self, index):
        path, target = self.samples[index]

        try:
            image = Image.open(path).convert('RGB')
            image = ImageOps.exif_transpose(image)

            if self.face_detector:
                img_np = np.array(image)
                results = self.face_detector(img_np, verbose=False)[0]

                if results.boxes is not None and len(results.boxes) > 0:
                    boxes = results.boxes.xyxy.cpu().numpy()
                    largest = max(boxes, key=lambda b: (b[2] - b[0]) * (b[3] - b[1]))
                    x1, y1, x2, y2 = map(int, largest)

                    W, H = image.size
                    x1, x2 = sorted((max(0, x1), min(W, x2)))
                    y1, y2 = sorted((max(0, y1), min(H, y2)))
                    image = image.crop((x1, y1, x2, y2))
                else:
                    return None  # 얼굴 검출 실패 → 스킵
        except Exception as e:
            print(f"❌ 이미지 로딩 실패: {path} / {e}")
            return None

        if self.transform:
            image = self.transform(image)

        return image, target

def collate_fn_remove_none(batch):
    # None이 아닌 샘플만 모아서 튜플로 반환
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return torch.empty(0), torch.empty(0)
    return tuple(zip(*batch))


# 코드셀 2 ResNet의 기본 블록 정의 (BasicBlock)

In [3]:
import torch
import torch.nn as nn


class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)

# 코드셀 3  Emotion-FAN의 전체 모델 구조 (ResNet + Attention) 정의

In [5]:
import torch
import torch.nn as nn

class ResNet_AT_Attention(nn.Module):
    def __init__(self, block, layers, num_classes=4, at_type='self-attention', dropout_rate=0.3):
        super().__init__()
        self.inplanes = 64
        self.at_type = at_type

        # 기본 ResNet18 구조
        self.conv1 = nn.Conv2d(3, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], 2)
        self.layer3 = self._make_layer(block, 256, layers[2], 2)
        self.layer4 = self._make_layer(block, 512, layers[3], 2)

        # Dropout layers 추가 (각 block 뒤에 삽입)
        self.dropout1 = nn.Dropout(dropout_rate)  # layer2 뒤
        self.dropout2 = nn.Dropout(dropout_rate)  # layer3 뒤
        self.dropout3 = nn.Dropout(dropout_rate)  # layer4 뒤

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_rate)  # 기존 FC 앞 dropout 유지

        # attention 계산용 α
        self.alpha = nn.Sequential(
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

        # 감정 예측용 FC
        self.pred_fc1 = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, 1, stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        layers = [block(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x=None, phrase="eval", AT_level="first_level", vm=None):
        if phrase == "eval" and AT_level == "first_level":
            x = self.relu(self.bn1(self.conv1(x)))
            x = self.maxpool(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.dropout1(x)  # ⬅ dropout1 추가
            x = self.layer3(x)
            x = self.dropout2(x)  # ⬅ dropout2 추가
            x = self.layer4(x)
            x = self.dropout3(x)  # ⬅ dropout3 추가
            x = self.avgpool(x).squeeze(-1).squeeze(-1)  # shape: (B, 512)
            alpha = self.alpha(self.dropout(x))  # shape: (B, 1)
            return x, alpha

        elif phrase == "eval" and AT_level == "pred":
            return self.pred_fc1(self.dropout(vm))

        else:
            raise ValueError("Invalid phrase or AT_level")


# 코드셀 3-1   입력 정규화

In [7]:
from PIL import Image
import random
import torch
from torchvision import transforms
import numpy as np
import os
from ultralytics import YOLO

# ✅ 정규화 계산 함수
def compute_crop_mean_std(image_root, face_detector, sample_size=5000, resize=(224, 224)):
    transform_temp = transforms.Compose([
        transforms.Resize(resize),
        transforms.ToTensor()
    ])

    valid_exts = ('.jpg', '.jpeg', '.png')
    image_paths = []

    for class_folder in os.listdir(image_root):
        class_path = os.path.join(image_root, class_folder)
        if not os.path.isdir(class_path):
            continue
        for fname in os.listdir(class_path):
            if fname.lower().endswith(valid_exts):
                image_paths.append(os.path.join(class_path, fname))

    sampled_paths = random.sample(image_paths, min(sample_size, len(image_paths)))

    img_mean = torch.zeros(3)
    img_std = torch.zeros(3)
    valid_count = 0

    print(f"🔍 총 샘플 수: {len(sampled_paths)}")

    for i, path in enumerate(sampled_paths, 1):
        img = Image.open(path).convert('RGB')
        img = ImageOps.exif_transpose(img)
        img_np = np.array(img)

        results = face_detector(img_np, verbose=False)[0]
        if results.boxes is not None and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            x1, y1, x2, y2 = map(int, max(boxes, key=lambda b: (b[2]-b[0]) * (b[3]-b[1])))

            W, H = img.size
            x1, x2 = sorted((max(0, x1), min(W, x2)))
            y1, y2 = sorted((max(0, y1), min(H, y2)))

            face_pil = img.crop((x1, y1, x2, y2))
            tensor = transform_temp(face_pil)

            img_mean += tensor.mean(dim=(1, 2))
            img_std += tensor.std(dim=(1, 2))
            valid_count += 1

        # 진행률 출력
        print(f"\r✅ 진행률: {i}/{len(sampled_paths)} | 검출 성공: {valid_count}", end='')

    print()  # 줄 바꿈

    if valid_count == 0:
        raise ValueError("❌ 얼굴 검출에 성공한 이미지가 없습니다.")

    img_mean /= valid_count
    img_std /= valid_count

    print("📊 얼굴 crop 기준 정규화 정보")
    print("Mean:", img_mean)
    print("Std: ", img_std)

    return img_mean, img_std





In [17]:
image_root = r"C:\Users\SCK\Desktop\affectnet_split\train"
#image_root = r"C:\Users\SCK\Desktop\Data\img\train"
img_mean, img_std = compute_crop_mean_std(image_root, yolo_model)

🔍 총 샘플 수: 5000
✅ 진행률: 5000/5000 | 검출 성공: 4923
📊 얼굴 crop 기준 정규화 정보
Mean: tensor([0.6055, 0.4580, 0.3965])
Std:  tensor([0.2196, 0.1959, 0.1853])


# 코드셀 4 데이터 준비 + Optimizer, Loss, Scheduler 설정

In [9]:
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

# ✅ 클래스별 가중치 (튜닝이 없을 때 기본값)
class_weights = [1.5, 1.0, 1.0, 1.3]  # anger, happy, panic, sadness

# ✅ Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        p = torch.exp(logp)
        logp = (1 - p) ** self.gamma * logp

        if self.alpha is not None:
            alpha = torch.tensor(self.alpha).to(input.device)
            logp = alpha[target] * logp.gather(1, target.unsqueeze(1))

        loss = -logp.gather(1, target.unsqueeze(1)).squeeze()
        return loss.mean() if self.reduction == 'mean' else loss.sum()

# ✅ 데이터셋 로딩 함수
def get_data_loaders(train_dir, val_dir, img_mean, img_std, face_detector, batch_size):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=img_mean.tolist(), std=img_std.tolist())
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=img_mean.tolist(), std=img_std.tolist())
    ])

    train_dataset = CustomEmotionFolder(train_dir, transform=train_transform, face_detector=face_detector)
    val_dataset   = CustomEmotionFolder(val_dir, transform=val_transform, face_detector=face_detector)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn_remove_none  # ✅ 이거 추가
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn_remove_none  # ✅ 이거 추가
    )
    
    return train_loader, val_loader

# ✅ 학습 구성 함수 (Optuna 결과 반영 가능하도록 수정됨)
def get_training_components(model,
                            lr=1e-3,
                            weight_decay=1e-4,
                            label_smooth=0.1,
                            use_focal=True,
                            gamma=2.0,
                            alpha=None):

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr,
        weight_decay=weight_decay
    )

    if use_focal:
        # alpha가 None이거나 list가 아니면 기본값 사용
        if alpha is None or not isinstance(alpha, (list, tuple)):
            alpha = class_weights
        criterion = FocalLoss(gamma=gamma, alpha=alpha)
    else:
        criterion = nn.CrossEntropyLoss(label_smoothing=label_smooth)

    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=2
    )

    return optimizer, criterion, scheduler

# 코드셀 5 하이퍼 파라미터 튜닝

In [11]:
import optuna
from optuna.trial import Trial
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
import torch.nn.functional as F
from torch.utils.data import Subset
import numpy as np
from tqdm import tqdm
import random

# ✅ 사전학습 가중치 경로
weight_path = r"C:\Users\SCK\Emotion_FAN\pretrain_model\Resnet18_FER+_pytorch.pth.tar"

# ✅ 사전학습 백본 로드 함수
def load_pretrained_backbone(model, weight_path):
    ckpt = torch.load(weight_path, map_location="cpu", weights_only=False)
    state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
    cleaned = {k.replace("module.", ""): v for k, v in state_dict.items()}
    filtered = {k: v for k, v in cleaned.items() if k in model.state_dict() and 'pred_fc1' not in k and 'alpha' not in k}
    model.load_state_dict(filtered, strict=False)
    print("✅ 사전학습 가중치 로드 완료 (fc 제외)")

# ✅ Optuna 목적 함수
def objective(trial: Trial, face_detector, img_mean, img_std):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 🔧 하이퍼파라미터 탐색
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout', 0.3, 0.7)
    gamma = trial.suggest_float('gamma', 1.0, 5.0)
    label_smooth = trial.suggest_float("label_smoothing", 0.0, 0.2)
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "adamw", "sgd"])
    use_focal = trial.suggest_categorical("use_focal", [True, False])

    # ✅ 클래스별 alpha 튜닝 (4개 클래스: anger, happy, panic, sadness)
    alpha = [
        trial.suggest_float('alpha_anger',   0.5, 2.0),
        trial.suggest_float('alpha_happy',   0.5, 2.0),
        trial.suggest_float('alpha_panic',   0.5, 2.0),
        trial.suggest_float('alpha_sadness', 0.5, 2.0)
    ]

    # ✅ 모델 구성 및 백본 로딩
    model = ResNet_AT_Attention(BasicBlock, [2, 2, 2, 2], num_classes=4, dropout_rate=dropout_rate).to(device)
    load_pretrained_backbone(model, weight_path)

    for param in model.parameters():
        param.requires_grad = False
    for name, module in model.named_children():
        if name in ['layer2', 'layer3', 'layer4', 'pred_fc1', 'alpha']:
            for param in module.parameters():
                param.requires_grad = True

    # ✅ 훈련용 Subset 구성
    train_dir = r"C:\Users\SCK\Desktop\affectnet_split\train"
    batch_size = 32
    subset_size = 500

    full_dataset = CustomEmotionFolder(train_dir, transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=img_mean.tolist(), std=img_std.tolist())
    ]), face_detector=face_detector)

    labels = [full_dataset[i][1] for i in range(len(full_dataset))]
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_size / len(full_dataset), random_state=42)
    indices, _ = next(sss.split(np.zeros(len(labels)), labels))
    subset = Subset(full_dataset, indices)

    train_loader = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=True)

    # ✅ 옵티마이저 구성
    params = filter(lambda p: p.requires_grad, model.parameters())
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'adamw':
        optimizer = torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.SGD(params, lr=lr, weight_decay=weight_decay, momentum=0.9)

    # ✅ 손실 함수 구성
    if use_focal:
        criterion = FocalLoss(gamma=gamma, alpha=alpha)
    else:
        criterion = torch.nn.CrossEntropyLoss(label_smoothing=label_smooth)

    # ✅ 학습 및 평가 (간단하게 2 epoch)
    model.train()
    all_preds, all_labels = [], []

    for epoch in range(1):
        for images, labels in tqdm(train_loader, desc=f"Tuning Epoch {epoch+1}", leave=False):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            features, _ = model(images, phrase="eval", AT_level="first_level")
            outputs = model(None, phrase="eval", AT_level="pred", vm=features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')
    return f1



In [15]:
# ✅ Optuna 스터디 생성 및 실행
study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1)
)
study.optimize(lambda trial: objective(trial, face_detector, img_mean, img_std), n_trials=100)

# ✅ 결과 출력
print("🎯 Best params:", study.best_params)
print("📊 Best Macro F1:", study.best_value)

best_params = study.best_params

[I 2025-06-30 09:44:53,086] A new study created in memory with name: no-name-5d7b04bb-93de-43f2-9794-3732cb6c0dd6


✅ 사전학습 가중치 로드 완료 (fc 제외)
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}


[I 2025-06-30 10:01:25,228] Trial 0 finished with value: 0.8456349645625636 and parameters: {'lr': 0.0002547878584517958, 'weight_decay': 1.1019323329282125e-06, 'dropout': 0.49274433239122695, 'gamma': 2.474464374623031, 'alpha': 0.3340506815773031, 'label_smoothing': 0.019861874243776237, 'optimizer': 'adamw', 'use_focal': True}. Best is trial 0 with value: 0.8456349645625636.


✅ 사전학습 가중치 로드 완료 (fc 제외)
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}


[I 2025-06-30 10:17:35,632] Trial 1 finished with value: 0.8532772262606967 and parameters: {'lr': 0.0005599628108249521, 'weight_decay': 0.0017184645830374244, 'dropout': 0.37647491631235813, 'gamma': 3.047649275789956, 'alpha': 0.9361859197427009, 'label_smoothing': 0.12817079144991153, 'optimizer': 'adamw', 'use_focal': True}. Best is trial 1 with value: 0.8532772262606967.


...

✅ 사전학습 가중치 로드 완료 (fc 제외)
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}


[I 2025-06-30 12:55:39,613] Trial 11 finished with value: 0.790110695778403 and parameters: {'lr': 5.111267349846751e-05, 'weight_decay': 0.0011304987635155346, 'dropout': 0.3955339736500943, 'gamma': 3.3574008395434123, 'alpha': 0.3754937591698184, 'label_smoothing': 0.15052181403116893, 'optimizer': 'adamw', 'use_focal': True}. Best is trial 1 with value: 0.8532772262606967.


✅ 사전학습 가중치 로드 완료 (fc 제외)
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}


[W 2025-06-30 13:05:05,697] Trial 12 failed with parameters: {'lr': 4.031370286578268e-05, 'weight_decay': 0.0007253513448048919, 'dropout': 0.39104248463533764, 'gamma': 3.027774636640373, 'alpha': 0.7833953625858975, 'label_smoothing': 0.14369746030472666, 'optimizer': 'adam', 'use_focal': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\SCK\AppData\Roaming\Python\Python312\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\SCK\AppData\Local\Temp\ipykernel_6024\3875402055.py", line 116, in objective
    features, _ = model(images, phrase="eval", AT_level="first_level")
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SCK\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^

KeyboardInterrupt: 

In [ ]:
import json

# best_params만 저장
with open("best_hyperparams.json", "w") as f:
    json.dump(study.best_params, f, indent=4)

In [13]:
# ✅ 사전학습 가중치 경로
weight_path = r"C:\Users\SCK\Emotion_FAN\pretrain_model\Resnet18_FER+_pytorch.pth.tar"

# ✅ 사전학습 백본 로드 함수
def load_pretrained_backbone(model, weight_path):
    ckpt = torch.load(weight_path, map_location="cpu", weights_only=False)
    state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
    cleaned = {k.replace("module.", ""): v for k, v in state_dict.items()}
    filtered = {k: v for k, v in cleaned.items() if k in model.state_dict() and 'pred_fc1' not in k and 'alpha' not in k}
    model.load_state_dict(filtered, strict=False)
    print("✅ 사전학습 가중치 로드 완료 (fc 제외)")

# 저장된 JSON 파일 경로
load_path = "best_hyperparams.json"

# 불러오기
with open(load_path, "r") as f:
    best_params = json.load(f)

print("✅ 불러온 하이퍼파라미터:")
print(best_params)


✅ 불러온 하이퍼파라미터:
{'lr': 0.00035535733759076474, 'weight_decay': 0.00012145648201289749, 'dropout': 0.30046245637123054, 'gamma': 1.474881640847121, 'label_smoothing': 0.02511914616985348, 'optimizer': 'adamw', 'alpha_anger': 0.7001358928255927, 'alpha_happy': 1.145787238039491, 'alpha_panic': 1.0260425562585442, 'alpha_sadness': 0.9407545081321703}


# 코드셀 6 - 튜닝 반영

In [19]:
# ✅ 튜닝 결과 존재 확인
assert 'best_params' in globals(), "Optuna 튜닝 결과가 없습니다."

# ✅ 하이퍼파라미터 추출
batch_size     = best_params.get('batch_size', 32)
dropout        = best_params['dropout']
lr             = best_params['lr']
weight_decay   = best_params['weight_decay']
label_smooth   = best_params.get('label_smoothing', 0.1)
use_focal      = best_params.get('use_focal', True)
gamma          = best_params.get('gamma', 2.0)

# ✅ 클래스별 alpha 가중치 튜닝값
alpha = [
    best_params.get('alpha_anger',   1.5),
    best_params.get('alpha_happy',   1.0),
    best_params.get('alpha_panic',   1.0),
    best_params.get('alpha_sadness', 1.3)
]

# ✅ 경로 설정
train_dir = r"C:\Users\SCK\Desktop\affectnet_split\train"
val_dir   = r"C:\Users\SCK\Desktop\affectnet_split\val"
weight_path = r"C:\Users\SCK\Emotion_FAN\pretrain_model\Resnet18_FER+_pytorch.pth.tar"

# ✅ YOLOv8 얼굴 탐지기 로드
face_detector = YOLO("yolov8n-face.pt")

# ✅ 데이터 로더 구성 (YOLO 기반 얼굴 감지 포함)
train_loader, val_loader = get_data_loaders(
    train_dir=train_dir,
    val_dir=val_dir,
    img_mean=img_mean,  # 전역 변수
    img_std=img_std,    # 전역 변수
    face_detector=face_detector,
    batch_size=batch_size
)

# ✅ 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 모델 초기화 및 백본 로드
model = ResNet_AT_Attention(BasicBlock, [2, 2, 2, 2], num_classes=4, dropout_rate=dropout).to(device)
load_pretrained_backbone(model, weight_path)

# ✅ 학습할 레이어만 설정
for param in model.parameters():
    param.requires_grad = False
for name, module in model.named_children():
    if name in ['layer2', 'layer3', 'layer4', 'pred_fc1', 'alpha']:
        for param in module.parameters():
            param.requires_grad = True

# ✅ 옵티마이저, 손실함수, 스케줄러 구성
optimizer, criterion, scheduler = get_training_components(
    model=model,
    lr=lr,
    weight_decay=weight_decay,
    label_smooth=label_smooth,
    use_focal=use_focal,
    gamma=gamma,
    alpha=alpha
)

print("✅ YOLOv8 기반 전체 학습 구성 완료")

✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}
✅ 사전학습 가중치 로드 완료 (fc 제외)
✅ YOLOv8 기반 전체 학습 구성 완료


# 코드셀 7 학습/검증 루프 함수 및 모델 저장 함수 정의

In [21]:
import os
import torch
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss, total_correct = 0.0, 0
    total_samples = 0
    all_preds, all_labels = [], []

    for images, labels in tqdm(dataloader, desc="Train", leave=False):
        if len(images) == 0:  # ✅ 빈 배치 예외 처리
            continue

        images = torch.stack(images).to(device)      # ✅ tuple → tensor
        labels = torch.tensor(labels).to(device)     # ✅ list/tuple → tensor

        optimizer.zero_grad()
        features, _ = model(images, phrase="eval", AT_level="first_level")
        outputs = model(None, phrase="eval", AT_level="pred", vm=features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total_loss += loss.item()
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    class_f1 = f1_score(all_labels, all_preds, average=None)

    return avg_loss, accuracy, macro_f1, class_f1


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_correct = 0.0, 0
    total_samples = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Val", leave=False):
            if len(images) == 0:  # ✅ 빈 배치 예외 처리
                continue

            images = torch.stack(images).to(device)
            labels = torch.tensor(labels).to(device)

            features, _ = model(images, phrase="eval", AT_level="first_level")
            outputs = model(None, phrase="eval", AT_level="pred", vm=features)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total_loss += loss.item()
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    class_f1 = f1_score(all_labels, all_preds, average=None)

    return avg_loss, accuracy, macro_f1, class_f1


def save_model(model, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f"💾 모델 저장 완료: {save_path}")


# 코드셀 8  전체 학습 루프 실행 (Epoch 반복 + Early Stopping + 모델 저장)

In [21]:
# 학습 설정
num_epochs = 10
best_val_acc = 0.0
save_path = r"C:\Users\SCK\checkpoints\ea_emotionfan_with_attention.pth"
patience = 3
patience_counter = 0

labels = ['happy', 'sadness', 'anger', 'panic']

for epoch in range(1, num_epochs + 1):
    print(f"\n🌀 Epoch {epoch}/{num_epochs}")
    
    train_loss, train_acc, train_f1, train_class_f1 = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_f1, val_class_f1 = validate(model, val_loader, criterion, device)

    print(f"📈 Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Macro F1: {train_f1:.4f}")
    print(f"📉 Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Macro F1: {val_f1:.4f}")

    # 클래스별 F1 출력
    print("📄 [클래스별 F1-score - Train]")
    for i, score in enumerate(train_class_f1):
        print(f"  {labels[i]}: {score:.4f}")
    
    print("📄 [클래스별 F1-score - Val]")
    for i, score in enumerate(val_class_f1):
        print(f"  {labels[i]}: {score:.4f}")

    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        save_model(model, save_path)
    else:
        patience_counter += 1
        print(f"⏳ EarlyStopping 대기... {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("⛔ Early stopping triggered!")
            break


🌀 Epoch 1/10


📈 Train Loss: 0.2256 | Acc: 0.8288 | Macro F1: 0.8207
📉 Val   Loss: 0.1375 | Acc: 0.8871 | Macro F1: 0.8838
📄 [클래스별 F1-score - Train]
  happy: 0.8080
  sadness: 0.9137
  anger: 0.8433
  panic: 0.7178
📄 [클래스별 F1-score - Val]
  happy: 0.8639
  sadness: 0.9437
  anger: 0.8912
  panic: 0.8365
💾 모델 저장 완료: C:\Users\SCK\checkpoints\ea_emotionfan_with_attention.pth

🌀 Epoch 2/10


📈 Train Loss: 0.1547 | Acc: 0.8678 | Macro F1: 0.8623
📉 Val   Loss: 0.1311 | Acc: 0.8816 | Macro F1: 0.8765
📄 [클래스별 F1-score - Train]
  happy: 0.8483
  sadness: 0.9357
  anger: 0.8764
  panic: 0.7888
📄 [클래스별 F1-score - Val]
  happy: 0.8549
  sadness: 0.9435
  anger: 0.8866
  panic: 0.8211
⏳ EarlyStopping 대기... 1/3

🌀 Epoch 3/10


📈 Train Loss: 0.1355 | Acc: 0.8848 | Macro F1: 0.8795
📉 Val   Loss: 0.1427 | Acc: 0.8758 | Macro F1: 0.8727
📄 [클래스별 F1-score - Train]
  happy: 0.8652
  sadness: 0.9455
  anger: 0.8972
  panic: 0.8100
📄 [클래스별 F1-score - Val]
  happy: 0.8609
  sadness: 0.9320
  anger: 0.8794
  panic: 0.8184
⏳ EarlyStopping 대기... 2/3

🌀 Epoch 4/10


📈 Train Loss: 0.1225 | Acc: 0.8931 | Macro F1: 0.8887
📉 Val   Loss: 0.1359 | Acc: 0.8819 | Macro F1: 0.8723
📄 [클래스별 F1-score - Train]
  happy: 0.8733
  sadness: 0.9489
  anger: 0.9008
  panic: 0.8319
📄 [클래스별 F1-score - Val]
  happy: 0.8699
  sadness: 0.9543
  anger: 0.8908
  panic: 0.7744
⏳ EarlyStopping 대기... 3/3
⛔ Early stopping triggered!


# 코드셀 9 - 커스텀 데이터셋 정규화

In [23]:
import json

# ✅ 커스텀 데이터 경로
custom_image_root = r"C:\Users\SCK\Desktop\Data\img\train"

# ✅ 정규화 정보 계산 함수 재사용
custom_img_mean, custom_img_std = compute_crop_mean_std(custom_image_root, face_detector=face_detector)

# ✅ 출력 예시
print("🔍 커스텀 Mean:", custom_img_mean)
print("🔍 커스텀 Std : ", custom_img_std)




norm_stats = {
    "mean": custom_img_mean.tolist(),
    "std": custom_img_std.tolist()
}

with open("custom_norm.json", "w") as f:
    json.dump(norm_stats, f, indent=4)

print("✅ Saved custom_norm.json:", norm_stats)

🔍 총 샘플 수: 5000
✅ 진행률: 5000/5000 | 검출 성공: 4999
📊 얼굴 crop 기준 정규화 정보
Mean: tensor([0.5718, 0.4455, 0.3920])
Std:  tensor([0.2276, 0.1906, 0.1701])
🔍 커스텀 Mean: tensor([0.5718, 0.4455, 0.3920])
🔍 커스텀 Std :  tensor([0.2276, 0.1906, 0.1701])
✅ Saved custom_norm.json: {'mean': [0.5717952847480774, 0.4455448091030121, 0.39195016026496887], 'std': [0.22755862772464752, 0.19057472050189972, 0.17012467980384827]}


# 코드셀 10 - 커스텀 데이터 셋 로드

In [24]:
# ✅ 커스텀 데이터셋 경로 설정
custom_train_dir = r"C:\Users\SCK\Desktop\Data\img\train"
custom_val_dir   = r"C:\Users\SCK\Desktop\Data\img\val"


# ✅ DataLoader 불러오기 (함수 재사용)
custom_train_loader, custom_val_loader = get_data_loaders(
    train_dir=custom_train_dir,
    val_dir=custom_val_dir,
    img_mean=custom_img_mean,
    img_std=custom_img_std,
    face_detector=face_detector,
    batch_size=32
)

✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}
✅ class_to_idx: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}


# 코드셀 11 - 모델 이어받기

In [25]:
# ✅ Fine-tuning 설정
finetune_epochs = 30
finetune_best_val_acc = 0.0
finetune_patience = 5
finetune_counter = 0

labels = ['happy', 'sadness', 'anger', 'panic']  # 클래스별 F1 출력용 이름

# 🔁 Fine-tuning Epoch Loop
for epoch in range(1, finetune_epochs + 1):
    print(f"\n🔁 [Fine-tuning] Epoch {epoch}/{finetune_epochs}")

    # ⬇ 학습 및 검증 (클래스별 F1 포함)
    train_loss, train_acc, train_f1, train_class_f1 = train_one_epoch(
        model, custom_train_loader, optimizer, criterion, device
    )
    val_loss, val_acc, val_f1, val_class_f1 = validate(
        model, custom_val_loader, criterion, device
    )

    # ✅ 전체 성능 지표 출력
    print(f"📈 [Train] Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Macro F1: {train_f1:.4f}")
    print(f"📉 [Val]   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Macro F1: {val_f1:.4f}")

    # ✅ 클래스별 F1-score 출력
    print("📄 [클래스별 F1-score - Train]")
    for i, score in enumerate(train_class_f1):
        print(f"  {labels[i]}: {score:.4f}")
    
    print("📄 [클래스별 F1-score - Val]")
    for i, score in enumerate(val_class_f1):
        print(f"  {labels[i]}: {score:.4f}")

    # ✅ 스케줄러 업데이트 (val_acc 기준)
    scheduler.step(val_acc)

    # ✅ 모델 저장 조건 및 EarlyStopping
    if val_acc > finetune_best_val_acc:
        finetune_best_val_acc = val_acc
        finetune_counter = 0
        save_model(model, r"C:\Users\SCK\checkpoints\ea_emotionfan_finetuned_custom.pth")
    else:
        finetune_counter += 1
        print(f"⏳ Fine-tuning EarlyStopping 대기... {finetune_counter}/{finetune_patience}")
        if finetune_counter >= finetune_patience:
            print("⛔ Fine-tuning 조기 종료!")
            break



🔁 [Fine-tuning] Epoch 1/30


📈 [Train] Loss: 0.2965 | Acc: 0.7960 | Macro F1: 0.7956
📉 [Val]   Loss: 0.2334 | Acc: 0.8300 | Macro F1: 0.8295
📄 [클래스별 F1-score - Train]
  happy: 0.7089
  sadness: 0.9126
  anger: 0.7612
  panic: 0.7996
📄 [클래스별 F1-score - Val]
  happy: 0.7786
  sadness: 0.9472
  anger: 0.7692
  panic: 0.8231
💾 모델 저장 완료: C:\Users\SCK\checkpoints\ea_emotionfan_finetuned_custom.pth

🔁 [Fine-tuning] Epoch 2/30


📈 [Train] Loss: 0.2023 | Acc: 0.8534 | Macro F1: 0.8530
📉 [Val]   Loss: 0.2747 | Acc: 0.8167 | Macro F1: 0.8161
📄 [클래스별 F1-score - Train]
  happy: 0.7962
  sadness: 0.9458
  anger: 0.8229
  panic: 0.8469
📄 [클래스별 F1-score - Val]
  happy: 0.7310
  sadness: 0.9474
  anger: 0.7822
  panic: 0.8040
⏳ Fine-tuning EarlyStopping 대기... 1/5

🔁 [Fine-tuning] Epoch 3/30


📈 [Train] Loss: 0.1686 | Acc: 0.8729 | Macro F1: 0.8727
📉 [Val]   Loss: 0.3279 | Acc: 0.8050 | Macro F1: 0.8031
📄 [클래스별 F1-score - Train]
  happy: 0.8195
  sadness: 0.9547
  anger: 0.8464
  panic: 0.8703
📄 [클래스별 F1-score - Val]
  happy: 0.7670
  sadness: 0.9524
  anger: 0.7580
  panic: 0.7351
⏳ Fine-tuning EarlyStopping 대기... 2/5

🔁 [Fine-tuning] Epoch 4/30


📈 [Train] Loss: 0.1503 | Acc: 0.8817 | Macro F1: 0.8816
📉 [Val]   Loss: 0.2885 | Acc: 0.8167 | Macro F1: 0.8150
📄 [클래스별 F1-score - Train]
  happy: 0.8292
  sadness: 0.9572
  anger: 0.8552
  panic: 0.8847
📄 [클래스별 F1-score - Val]
  happy: 0.7702
  sadness: 0.9555
  anger: 0.7867
  panic: 0.7475
⏳ Fine-tuning EarlyStopping 대기... 3/5

🔁 [Fine-tuning] Epoch 5/30


...

📈 [Train] Loss: 0.0290 | Acc: 0.9763 | Macro F1: 0.9763
📉 [Val]   Loss: 0.3910 | Acc: 0.8400 | Macro F1: 0.8394
📄 [클래스별 F1-score - Train]
  happy: 0.9656
  sadness: 0.9930
  anger: 0.9710
  panic: 0.9757
📄 [클래스별 F1-score - Val]
  happy: 0.7890
  sadness: 0.9402
  anger: 0.8027
  panic: 0.8256
⏳ Fine-tuning EarlyStopping 대기... 1/5

🔁 [Fine-tuning] Epoch 16/30


📈 [Train] Loss: 0.0272 | Acc: 0.9775 | Macro F1: 0.9775
📉 [Val]   Loss: 0.3656 | Acc: 0.8367 | Macro F1: 0.8363
📄 [클래스별 F1-score - Train]
  happy: 0.9676
  sadness: 0.9940
  anger: 0.9710
  panic: 0.9773
📄 [클래스별 F1-score - Val]
  happy: 0.7778
  sadness: 0.9521
  anger: 0.7974
  panic: 0.8182
⏳ Fine-tuning EarlyStopping 대기... 2/5

🔁 [Fine-tuning] Epoch 17/30


📈 [Train] Loss: 0.0303 | Acc: 0.9748 | Macro F1: 0.9748
📉 [Val]   Loss: 0.3685 | Acc: 0.8350 | Macro F1: 0.8347
📄 [클래스별 F1-score - Train]
  happy: 0.9659
  sadness: 0.9910
  anger: 0.9711
  panic: 0.9713
📄 [클래스별 F1-score - Val]
  happy: 0.7763
  sadness: 0.9587
  anger: 0.7891
  panic: 0.8147
⏳ Fine-tuning EarlyStopping 대기... 3/5

🔁 [Fine-tuning] Epoch 18/30


📈 [Train] Loss: 0.0239 | Acc: 0.9820 | Macro F1: 0.9820
📉 [Val]   Loss: 0.3783 | Acc: 0.8383 | Macro F1: 0.8380
📄 [클래스별 F1-score - Train]
  happy: 0.9746
  sadness: 0.9940
  anger: 0.9793
  panic: 0.9800
📄 [클래스별 F1-score - Val]
  happy: 0.7739
  sadness: 0.9539
  anger: 0.8045
  panic: 0.8196
⏳ Fine-tuning EarlyStopping 대기... 4/5

🔁 [Fine-tuning] Epoch 19/30


📈 [Train] Loss: 0.0210 | Acc: 0.9820 | Macro F1: 0.9820
📉 [Val]   Loss: 0.3837 | Acc: 0.8383 | Macro F1: 0.8383
📄 [클래스별 F1-score - Train]
  happy: 0.9760
  sadness: 0.9947
  anger: 0.9770
  panic: 0.9803
📄 [클래스별 F1-score - Val]
  happy: 0.7759
  sadness: 0.9565
  anger: 0.8019
  panic: 0.8191
⏳ Fine-tuning EarlyStopping 대기... 5/5
⛔ Fine-tuning 조기 종료!


# 코드셀 12 - test set 실행

In [28]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset

face_detector = YOLO("yolov8n-face.pt")

# ✅ 추론 전용 데이터셋
class InferenceEmotionDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, face_detector=None):
        super().__init__(root, transform=transform)
        self.face_detector = face_detector

    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            image = Image.open(path).convert("RGB")
            image = ImageOps.exif_transpose(image)

            if self.face_detector:
                img_np = np.array(image)
                results = self.face_detector(img_np, verbose=False)[0]
                if results.boxes is not None and len(results.boxes) > 0:
                    boxes = results.boxes.xyxy.cpu().numpy()
                    x1, y1, x2, y2 = map(int, max(boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1])))
                    W, H = image.size
                    x1, x2 = sorted((max(0, x1), min(W, x2)))
                    y1, y2 = sorted((max(0, y1), min(H, y2)))
                    image = image.crop((x1, y1, x2, y2))
                else:
                    print(f"⚠️ 얼굴 미검출 → 원본 이미지 사용: {os.path.basename(path)}")

            if self.transform:
                image = self.transform(image)

            return image, target  # ✅ label도 함께 반환

        except Exception as e:
            print(f"❌ 이미지 로딩 실패: {path} / {e}")
            return None

# ✅ collate_fn: None 제거
def collate_fn_remove_none(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return torch.empty(0), []
    return tuple(zip(*batch))

# ✅ 전처리 정의 (정규화 포함)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=custom_img_mean.tolist(), std=custom_img_std.tolist())
])

# ✅ 데이터셋 및 로더
test_root = r"C:\Users\SCK\Desktop\test\image"
test_dataset = InferenceEmotionDataset(test_root, transform=test_transform, face_detector=face_detector)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn_remove_none)

# ✅ 클래스 idx → 이름 매핑
idx2emotion = {v: k for k, v in test_dataset.class_to_idx.items()}

# ✅ 추론 클래스
class EmotionFANPredictor_Attention:
    def __init__(self, weight_path, at_type="self-attention", dropout_rate=0.3):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ResNet_AT_Attention(BasicBlock, [2, 2, 2, 2],
                                         num_classes=4,
                                         at_type=at_type,
                                         dropout_rate=dropout_rate).to(self.device)
        self.load_weights(weight_path)
        self.model.eval()

    def load_weights(self, ckpt_path):
        ckpt = torch.load(ckpt_path, map_location=self.device)
        state_dict = ckpt.get("state_dict", ckpt)
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        self.model.load_state_dict(state_dict, strict=False)

    def predict(self, img_tensor):
        img_tensor = img_tensor.to(self.device).unsqueeze(0)
        with torch.no_grad():
            f, alpha = self.model(img_tensor, phrase="eval", AT_level="first_level")
            logits = self.model(None, phrase="eval", AT_level="pred", vm=f)
            pred_idx = torch.argmax(logits, dim=1).item()
            return pred_idx, alpha.cpu().item()

# ✅ 모델 로드
weight_path = r"C:\Users\SCK\checkpoints\ea_emotionfan_finetuned_custom.pth"
predictor = EmotionFANPredictor_Attention(weight_path)

# ✅ 추론 실행
results = []
y_true = []  # (옵션: 레이블이 있는 경우만 사용)
y_pred = []

for batch in tqdm(test_loader, desc="🔍 Inference"):
    if len(batch) == 0:
        continue

    img_tensor, targets = batch  # filenames → targets (정수 인덱스)
    img_tensor = img_tensor[0]
    target = targets[0]

    pred_idx, _ = predictor.predict(img_tensor)
    y_pred.append(pred_idx)
    y_true.append(target)

    results.append({
        "filename": test_dataset.samples[len(y_true)-1][0],  # 또는 os.path.basename(...)
        "predicted_label": idx2emotion[pred_idx],
        "true_label": idx2emotion[target]
    })


# ✅ 평가 지표 출력 (정답 y_true가 있는 경우에만)
acc = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\n🎯 Accuracy: {acc:.4f}")
print(f"📊 Macro F1-score: {macro_f1:.4f}")
print("\n📄 [클래스별 F1-score]")
print(classification_report(y_true, y_pred, target_names=[idx2emotion[i] for i in sorted(idx2emotion.keys())]))

# ✅ JSON 저장
json_path = r"C:\Users\SCK\Desktop\test\prediction_results.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"\n📝 예측 결과가 JSON 파일로 저장되었습니다: {json_path}")

🔍 Inference: 100%|████████████████████████████████████████████████████████████████| 1200/1200 [02:08<00:00,  9.35it/s]


🎯 Accuracy: 0.8367
📊 Macro F1-score: 0.8351

📄 [클래스별 F1-score]
              precision    recall  f1-score   support

       anger       0.81      0.69      0.74       300
       happy       0.92      0.95      0.93       300
       panic       0.76      0.87      0.81       300
     sadness       0.86      0.84      0.85       300

    accuracy                           0.84      1200
   macro avg       0.84      0.84      0.84      1200
weighted avg       0.84      0.84      0.84      1200


📝 예측 결과가 JSON 파일로 저장되었습니다: C:\Users\SCK\Desktop\test\prediction_results.json
